In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import scienceplots
import os

# Using seaborn's style
plt.style.use('science')
width = 'thesis'

def set_size(width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

In [ ]:
wd = "C:\\Users\\simulationsFFlab\\Desktop\\Simulations_Thesis"
os.chdir(wd)

In [ ]:
# Search the current directory for folders with the word simulation_2023 in them
# These folders contain the data from the simulations
folders = [f for f in os.listdir() if 'simulation_2023' in f]

In [ ]:
REPLICAS = [1, 2, 3, 4, 5]
Dms = [
    "DM_0.01",
    "DM_0.05",
    "DM_0.1",
    "DM_0.25",
    "DM_0.5",
    "DM_0.75",
    "DM_1.0",
    "DM_3.0",
    "DM_5.0",
    "DM_10.0",
    ]
De_vals = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 25.0, 50.0, 75.0]
Des = ["DE_{}".format(i) for i in De_vals]

# Create a dictionary to store the dataframes
dfs = {}

magData = {}
for Dm in Dms:
    magData[Dm] = {}
    for De in Des:
        magData[Dm][De] = []

In [ ]:
# Filter out the folders by the Dm value
for dm in Dms:
    flds = [f for f in folders if dm in f]
    # Loop over all the folders
    for folder in flds:
        # Change the working directory to the folder
        os.chdir(wd + "\\" + folder)
        # List the files in the folder if the files end with .npz
        files = [f for f in os.listdir() if f.endswith('.npz')]
        # Sort the files by the third integer after splitting by underscores
        files.sort(key=lambda f: int((f.split('_')[2]).split('.')[0]))
        # Open the last npz file and store the rfp and gfp arrays in variables
        data = np.load(files[-1])
        RFP = data['rfp']
        GFP = data['gfp']
        # Create an array where if GFP > RFP the value is -1 and if GFP < RFP the value is 1
        # and if GFP = RFP the value is 0
        lattice = np.zeros((RFP.shape[0], RFP.shape[1]))
        lattice[GFP > RFP] = -1
        lattice[GFP < RFP] = 1
        # Calculate the magnetization of the lattice
        magnetization = np.sum(lattice)
        # Calculate the average magnetization of the lattice
        average_magnetization = magnetization/(RFP.shape[0]*RFP.shape[1])
        # Print the average magnetization and the folder name
        print(average_magnetization, folder)
        # Get the DE value from the folder name (last two strings after splitting by underscores)
        De = '_'.join([folder.split('_')[-2], folder.split('_')[-1]])
        # Append the average magnetization to the dataframe
        magData[dm][De].append(average_magnetization)
        # Change the working directory back to the main working directory
        os.chdir(wd)

In [ ]:
# For each Dm value loop over the De values, calculate the mean and standard deviation
# and store them in a dataframe
for dm in Dms:
    for de in Des:
        magData[dm][de] = np.array(magData[dm][de])
        magData[dm][de] = [np.mean(magData[dm][de]), np.std(magData[dm][de])]

In [ ]:
# Plot the mean and standard deviation of the magnetization as a function of De
#Create a colormap for the Dms
cmap = plt.cm.get_cmap('inferno_r', len(Dms))

plt.figure(figsize=set_size(width))
for dm in Dms:
    if dm == "DM_0.01":
        plt.errorbar(De_vals, [magData[dm][De][0] for De in Des], yerr=[magData[dm][De][1] for De in Des], fmt='o', capsize=5, label=dm, color=cmap(Dms.index(dm)))
    else:
        pass
plt.xlabel(r"$D_e$")
plt.ylabel(r"$M$")
plt.xscale('log')
# Legend to the outside of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
# Create a 3D surface plot of the magnetization as a function of De and Dm
# Create a meshgrid of the De and Dm values
De_mesh, Dm_mesh = np.meshgrid(De_vals, [float(dm.split('_')[-1]) for dm in Dms])

# Create a 2D array of the mean magnetization values
mean_mag = np.array([[magData[dm][de][0] for de in Des] for dm in Dms])
# Create a 2D array of the standard deviation of the magnetization values
std_mag = np.array([[magData[dm][de][1] for de in Des] for dm in Dms])

# Plot the 3D surface
fig = plt.figure(figsize=set_size(width, fraction=1, subplots=(1, 1)))
ax = fig.add_subplot(111, projection='3d')
# Plot the mean magnetization as a surface
ax.plot_surface(np.log10(De_mesh), np.log10(Dm_mesh), mean_mag, cmap='inferno')
ax.set_xlabel(r"$\log_{10}(D_e)$")
ax.set_ylabel(r"$\log_{10}(D_m)$")
ax.set_zlabel(r"$\overline{M}$")

# Set a title
ax.set_title("Magnetización promedio ($\overline{M}$)\nen función de $D_e$ y $D_m$")

# Draw a surface a z=0.5 magnetization
#ax.plot_surface(np.log10(De_mesh), np.log10(Dm_mesh), np.zeros(mean_mag.shape) + 0.5, color='black', alpha=0.25)

# Project contour profiles onto the surface
#ax.contour(np.log10(De_mesh), np.log10(Dm_mesh), mean_mag, zdir='x', offset=-2.1, cmap='coolwarm', levels=10, linewidths=1)
#ax.contour(np.log10(De_mesh), np.log10(Dm_mesh), mean_mag, zdir='y', offset=1, cmap='coolwarm', levels=10, linewidths=1)

# Change the view axis
ax.view_init(15, 300)

# Tight layout
plt.tight_layout()

# Save the figure as a pdf
plt.savefig("C:\\Users\\simulationsFFlab\\Desktop\\Simulations_Thesis\\THESIS_FIGURES\\3D_Magnetization.pdf", dpi=300)

In [ ]:
# Create a 3D surface plot of the magnetization as a function of De and Dm
# Create a meshgrid of the De and Dm values
De_mesh, Dm_mesh = np.meshgrid(De_vals, [float(dm.split('_')[-1]) for dm in Dms])
# Create a meshgrid of the mean magnetization values
mag_mesh = np.array([[magData[dm][de][0] for de in Des] for dm in Dms])
# Create a meshgrid of the standard deviation of the magnetization values
mag_std_mesh = np.array([[magData[dm][de][1] for de in Des] for dm in Dms])

# Plot a 2D heatmap of the mean magnetization as a function of De and Dm
plt.figure(figsize=set_size(width))
plt.pcolormesh(np.log10(De_mesh), np.log10(Dm_mesh), mag_mesh, cmap='inferno')
plt.colorbar(label=r"$|\overline{M}|$")
plt.xlabel(r"$\log_{10} (D_e)$")
plt.ylabel(r"$\log_{10} (D_m)$")
# Set a title
plt.title("Heatmap de magnetización promedio ($\overline{M}$)\nen función de $D_e$ y $D_m$")
plt.tight_layout()

# Save the figure as a pdf
plt.savefig("C:\\Users\\simulationsFFlab\\Desktop\\Simulations_Thesis\\THESIS_FIGURES\\Magnetization_Heatmap.pdf", dpi=300)

In [ ]:
10**0.5